## **PARTIE I : INSTALLATION DES VERSIONS COMPATIBLES SANS CONFLIT**


Il faut installer  **PyTorch version 2.7.1** avec **CUDA 11.8 (cu118)**, assurant la compatibilité avec la **GPU Tesla T4**.

In [ ]:
!pip uninstall -y torch torchvision torchaudio timm


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: timm 1.0.16
Uninstalling timm-1.0.16:
  Successfully uninstalled timm-1.0.16


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install timm


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/90

In [ ]:
#On vérifie si les bonnes versions ont été installées
import torch, timm, torchvision
print(torch.__version__, torch.version.cuda)
print(torch.cuda.get_device_name(0))


2.7.1+cu118 11.8
Tesla T4


On force CUDA a lancé les opérations de façon synchrone, autrement dit, chaque opération GPU attend d’être terminée avant de continuer, ce qui facilite le debugging (on attrape les erreurs au moment où elles se produisent).

In [ ]:

%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


#**PARTIE II: Suppression des doulons**

Dans notre dataset inital, certaines images se retrouvent dans plusieurs lcasses en même temps. Il donc faut supprimer les doublons d’images identifiés par leur nom afin d’éviter que le même fichier apparaisse plusieurs fois dans différentes classes du dataset.

In [ ]:
#On monte d'abord notre Drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#On décompresse ensuite notre dossier comprenant les images (étape que l'on peut sauter une fois quele dossier a été décompressé)

import zipfile

zip_path = "/content/drive/MyDrive/Paleo_projet/greekdataset.zip"
extract_path = "/content/drive/MyDrive/Paleo_projet/greekrealdataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



On supprime les doublons en regardant si des noms exacts de fichiers apparaissent dans des sous-dossiers (=des classes) différentes.

In [ ]:

import os

# Dossier source contenant les sous-dossiers de classes d'images
source_dir = "/content/drive/MyDrive/Paleo_projet/greekrealdataset"

seen_files = {}  # Dictionnaire pour garder la trace des fichiers déjà rencontrés

# Parcours des éléments dans le dossier source
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)  # Chemin complet vers la classe

    # Vérifie que c'est bien un dossier (une classe), sinon on ignore
    if not os.path.isdir(class_path):
        continue

    # Parcours des fichiers dans le dossier de la classe
    for filename in os.listdir(class_path):
        # On ne traite que les fichiers .jpg (ignorer autres formats)
        if not filename.lower().endswith('.jpg'):
            continue

        file_path = os.path.join(class_path, filename)  # Chemin complet vers le fichier image

        # Si le fichier a déjà été rencontré (doublon!!)
        if filename in seen_files:
            print(f"Doublon détecté : {filename} (supprimé de {class_path})")  # Message avertissment doublon
            os.remove(file_path)  # Suppression du doublon dans ce dossier
        else:
            # Premier fichier avec ce nom rencontré, on l'ajoute au dictionnaire
            seen_files[filename] = class_path  # On mémorise où ce fichier a été gardé
            print(f"Gardé : {filename} (dans {class_path})")  # On dit que doublon est bien conservé en singleton

print("Nettoyage terminé !")


Puis on procède à uen vérifciation un peu plus robuste : Pour supprimer les doublons dans le dataset, on calcule d’abord le hash MD5 de chaque image, qui est une empreinte numérique unique générée à partir du contenu du fichier. Si plusieurs fichiers ont le même hash MD5, cela signifie qu’ils sont identiques. On conserve alors une seule occurrence de chaque image et on supprime toutes les copies exactes restantes.

In [ ]:

import hashlib
from collections import defaultdict

def file_hash(filepath):
    """Retourne le hash MD5 d’un fichier."""
    hasher = hashlib.md5()
    with open(filepath, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

def remove_duplicate_images(base_dir):
    hash_map = defaultdict(list)

    # Parcourt toutes les images dans toutes les classes
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(".jpg"):
                full_path = os.path.join(root, file)
                h = file_hash(full_path)
                hash_map[h].append(full_path)

    # Analyse des doublons
    duplicates = {h: paths for h, paths in hash_map.items() if len(paths) > 1}

    total_images = sum(len(v) for v in hash_map.values())
    print(f"Nombre total d’images analysées : {total_images}")
    print(f"Nombre d’images uniques : {len(hash_map)}")

    if duplicates:
        print(f"{len(duplicates)} doublons exacts trouvés, suppression en cours...")
        for h, files in duplicates.items():
            # Garde la première occurrence, supprime les autres
            files_to_delete = files[1:]
            for fpath in files_to_delete:
                try:
                    os.remove(fpath)
                    print(f"Supprimé : {fpath}")
                except Exception as e:
                    print(f"Erreur suppression {fpath} : {e}")
        print("Suppression des doublons terminée.")
    else:
        print("Aucun doublon.")

# Exécuter sur ton dataset de base
base_dataset = "/content/drive/MyDrive/Paleo_projet/greekrealdataset"
remove_duplicate_images(base_dataset)


Nombre total d’images analysées : 956
Nombre d’images uniques : 938

⚠️ 16 doublons exacts trouvés, suppression en cours...
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Factional/4.13._Acclamations_of_the_Greens,_the_City_and_the_Blues_A.jpg
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Religious/4.21._1._Acclamations_for_Albinos,_clarissimus_2._Acclamation_for_the_city_J.jpg
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Prayers + prayer invocation/1.14._Prayer_of_Anastasios,_donor_B.jpg
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Place inscription/5.12._i._Fragment_of_verse._ii._Place_inscription_of_Synodios_B.jpg
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Religious/5.9._Building_dedication_to_Aphrodite_and_Hadrian_B.jpg
Supprimé : /content/drive/MyDrive/Paleo_projet/greekrealdataset/Dedication + dedication to demos + dedication to emperor donor/7.2._Dedicatory_poem.jpg
Supprimé : /content/

On procède à une ultime vérification: le code est similairement le même que celui d'au-dessus.

In [ ]:
import os
import hashlib
from collections import defaultdict

def file_hash(filepath):
    """Retourne le hash MD5 d’un fichier."""
    hasher = hashlib.md5()
    with open(filepath, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

def check_unique_images(base_dir):
    hash_map = defaultdict(list)

    # Parcourt toutes les images dans toutes les classes
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(".jpg"):
                full_path = os.path.join(root, file)
                h = file_hash(full_path)
                hash_map[h].append(full_path)

    # Analyse des doublons
    duplicates = {h: paths for h, paths in hash_map.items() if len(paths) > 1}

    total_images = sum(len(v) for v in hash_map.values())
    print(f"Nombre total d’images analysées : {total_images}")
    print(f"Nombre d’images uniques : {len(hash_map)}")

    if duplicates:
        print(f"{len(duplicates)} doublons exacts trouvés :")
        for h, files in duplicates.items():
            print(f"\n--- Hash {h} ---")
            for path in files:
                print(f" • {path}")
    else:
        print("Toutes les images sont uniques.")

# Exécuter sur ton dataset de base
base_dataset = "/content/drive/MyDrive/Paleo_projet/greekrealdataset"
check_unique_images(base_dataset)


## **PARTIE III : Split du jeu de données en train/val**
 Il s'agit ensuite de diviser notre jeu de données propre, sans doublons, en deux ensembles : un d'entraînement et un de validation. On divise nos données en 80% entraînement et 20% de validation. Chaque fichier est également renommé  un nom neutre afin d’éviter que le modèle Vision Transformer ne se base sur des titres explicites des fichiers pour la classification.

In [ ]:
#Importation des librairies nécessaires pour le split des données
import shutil
import random
from pathlib import Path
from sklearn.model_selection import train_test_split


In [ ]:


source_dir = "/content/drive/MyDrive/Paleo_projet/greekrealdataset"
output_dir = "/content/drive/MyDrive/Paleo_projet/dataset_split"
train_ratio = 0.8  # 80% pour l'entraînement

# Supprime le dossier de sortie s'il existe déjà (évite les doublons)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

# Crée les nouveaux dossiers
for split in ["train", "val"]:
    for class_name in os.listdir(source_dir):
        os.makedirs(os.path.join(output_dir, split, class_name), exist_ok=True)

# Séparation et renommage
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith('.jpg')]
    train_images, val_images = train_test_split(images, train_size=train_ratio, random_state=42)

    # Pour chaque image dans le train
    for idx, image in enumerate(train_images):
        src = os.path.join(class_path, image)
        dst_filename = f"train_{class_name}_{idx:04d}.jpg"
        dst = os.path.join(output_dir, "train", class_name, dst_filename)
        shutil.copyfile(src, dst)

    # Pour chaque image dans le val
    for idx, image in enumerate(val_images):
        src = os.path.join(class_path, image)
        dst_filename = f"val_{class_name}_{idx:04d}.jpg"
        dst = os.path.join(output_dir, "val", class_name, dst_filename)
        shutil.copyfile(src, dst)

print("Split propre avec renommage neutre terminé !")


Split propre avec renommage neutre terminé !


In [ ]:
#On vérifie que le split est effectué correctement
def count_images_in_dir(dir_path):
    count = 0
    for root, _, files in os.walk(dir_path):
        count += sum(f.lower().endswith('.jpg') for f in files)
    return count

train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")

nb_train = count_images_in_dir(train_dir)
nb_val = count_images_in_dir(val_dir)

print(f"Nombre total d'images dans 'train' : {nb_train}")
print(f"Nombre total d'images dans 'val'   : {nb_val}")


Nombre total d'images dans 'train' : 739
Nombre total d'images dans 'val'   : 198


Avant toute opération, il faut penser à supprimer le dossier .DS_Store si ce dernier apparait.

In [ ]:
!find /content/drive/MyDrive/Paleo_projet/dataset_split -name ".DS_Store" -delete


In [ ]:
#On vérifie bien qu'il n'existe plus

for root, dirs, files in os.walk("/content/drive/MyDrive/Paleo_projet/dataset_split"):
    for file in files:
        if file == ".DS_Store":
            print("Il reste ENCORE un .DS_Store dans :", os.path.join(root, file))


##**PARTIE III: ENTRAINEMENT DU MODELE VIT**



Le modèle utilisé est un **Vision Transformer (ViT)** pré-entraîné sur **ImageNet**.



###  Prétraitement des images

Avant d’être envoyées au modèle, les images subissent une série de transformations :

-  **Redimensionnement** à `224x224` pixels pour correspondre à la taille attendue par ViT.  
- **Conversion en tenseur** PyTorch (`ToTensor`) pour les rendre compatibles avec les calculs sur GPU.  
-  **Normalisation** avec la moyenne et l’écart-type d’ImageNet :  
  - Moyenne : `[0.485, 0.456, 0.406]`  
  - Écart-type : `[0.229, 0.224, 0.225]`  



### A. Gestion du déséquilibre des classes

Le dataset étant déséquilibré, des poids de classes sont calculés inversément proportionnels à la fréquence de chaque classe.  
Ces poids sont intégrés à la fonction de perte `CrossEntropyLoss` pour que le modèle accorde plus d’importance aux classes rares.



###B.  Modèle utilisé

- Modèle : `vit_base_patch16_224`, pré-entraîné sur ImageNet
- Capable d’extraire des caractéristiques visuelles robustes : formes, textures, motifs.
- La tête de classification a été modifiée pour produire 18 scores, correspondant à nos 18 classes.




###E. Entraînement & Validation

Le modèle est entraîné sur 10 epochs.
La validation est effectuée sur notre ensemble non vu, avec génération de métriques comme l’accuracy la précision le rappel et le F1-score.


In [ ]:
import timm, torch, torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:

from collections import Counter
import os
import glob
from sklearn.metrics import classification_report, accuracy_score


# ETAPE 1 : DEFINIR LES HYPERPARAMETRES
BATCH_SIZE  = 16           # On définit la taille du batch
IMG_SIZE    = 224          # On fixe la taille des images à 224x224 pixels
NUM_CLASSES = 17         # On précise le nombre de classes dans notre dataset

#ETAPE 2: PRETRAITEMENT DES IMAGES

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),    # On redimensionne toutes les images à la taille souhaitée
    transforms.ToTensor(),                       # On convertit les images en tenseurs PyTorch
    transforms.Normalize(                        # On normalise les images avec les moyennes et écarts types d'ImageNet (cette ligne a été implémentée par Gemini)
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

train_dir = "/content/drive/MyDrive/Paleo_projet/dataset_split/train"   # On indique le chemin vers le dossier des données d'entraînement
val_dir   = "/content/drive/MyDrive/Paleo_projet/dataset_split/val"     # On indique le chemin vers le dossier des données de validation
train_dataset = datasets.ImageFolder(train_dir, transform=transform)   # On charge les images d'entraînement avec leur transformation
val_dataset   = datasets.ImageFolder(val_dir,   transform=transform)   # On charge les images de validation avec leur transformation

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  # On crée un DataLoader pour l'entraînement avec mélange des données pour que le modèle voit des exemples variés à chaque batch
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE)                 # On crée un DataLoader pour la validation sans mélange



In [ ]:


# Il faut pondérer les classes puisque notre datasetr est très déséquilibré
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
targets = train_dataset.targets
class_counts = Counter(targets)
total_samples = sum(class_counts.values())
class_weights = [total_samples / class_counts[i] for i in range(NUM_CLASSES)]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("Poids des classes :", class_weights)

# ETAPE 3 : DEFINITION DU MODELE VIT
model = timm.create_model("vit_base_patch16_224", pretrained=True)   # On charge un modèle ViT pré-entraîné sur ImageNet
model.head = nn.Linear(model.head.in_features, NUM_CLASSES)          # On remplace la tête du modèle pour l'adapter à notre nombre de classes (18)

model.to(device)                                                      # On déplace le modèle sur le périphérique choisi

criterion = nn.CrossEntropyLoss(weight=class_weights)  # On définit la fonction de perte avec pondération des classes
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)  # On initialise l'optimiseur AdamW avec un taux d'apprentissage fixé

#+# ENTRAINEMENT DU MODELE VIT
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss : {running_loss/len(train_loader):.4f}")

    # --- phase validation ---
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}")
    print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

    model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Accuracy :", accuracy_score(all_labels, all_preds))
print("Classification Report :\n", classification_report(all_labels, all_preds, zero_division=0))



Poids des classes : tensor([ 23.0938, 184.7500,  10.8676,  52.7857,  18.4750, 147.8000,   4.2229,
         46.1875,  92.3750,   3.3288, 184.7500,  41.0556,  23.0938,  36.9500,
         12.9649,  36.9500, 184.7500], device='cuda:0')
Epoch 1 | Loss : 2.9721
Validation Accuracy: 0.0960
                                                                precision    recall  f1-score   support

                                                   Acclamation       0.18      0.78      0.30         9
                                                     Agonistic       0.00      0.00      0.00         2
                    Building dedication - building inscription       0.50      0.06      0.10        18
                                                        Decree       0.00      0.00      0.00         4
Dedication + dedication to demos + dedication to emperor donor       0.50      0.09      0.15        11
                                                     Factional       0.00      0.00      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2 | Loss : 2.7308
Validation Accuracy: 0.1566
                                                                precision    recall  f1-score   support

                                                   Acclamation       0.50      0.44      0.47         9
                                                     Agonistic       0.00      0.00      0.00         2
                    Building dedication - building inscription       0.24      0.22      0.23        18
                                                        Decree       0.00      0.00      0.00         4
Dedication + dedication to demos + dedication to emperor donor       0.14      0.09      0.11        11
                                                     Factional       0.33      0.50      0.40         2
                                     Funerary + funerary verse       0.50      0.09      0.15        44
                                Gladiator + Gladiator memorial       0.06      0.80      0.11         5
           

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 3 | Loss : 2.2281
Validation Accuracy: 0.2576
                                                                precision    recall  f1-score   support

                                                   Acclamation       0.80      0.44      0.57         9
                                                     Agonistic       0.00      0.00      0.00         2
                    Building dedication - building inscription       0.22      0.11      0.15        18
                                                        Decree       0.08      0.25      0.12         4
Dedication + dedication to demos + dedication to emperor donor       0.00      0.00      0.00        11
                                                     Factional       0.00      0.00      0.00         2
                                     Funerary + funerary verse       0.43      0.30      0.35        44
                                Gladiator + Gladiator memorial       0.18      1.00      0.30         5
           

...

Epoch 9 | Loss : 0.0036
Validation Accuracy: 0.4545
                                                                precision    recall  f1-score   support

                                                   Acclamation       0.67      0.44      0.53         9
                                                     Agonistic       0.00      0.00      0.00         2
                    Building dedication - building inscription       0.29      0.22      0.25        18
                                                        Decree       0.00      0.00      0.00         4
Dedication + dedication to demos + dedication to emperor donor       1.00      0.09      0.17        11
                                                     Factional       0.50      0.50      0.50         2
                                     Funerary + funerary verse       0.43      0.59      0.50        44
                                Gladiator + Gladiator memorial       1.00      0.40      0.57         5
           

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 10 | Loss : 0.0026
Validation Accuracy: 0.4545
                                                                precision    recall  f1-score   support

                                                   Acclamation       0.67      0.44      0.53         9
                                                     Agonistic       0.00      0.00      0.00         2
                    Building dedication - building inscription       0.29      0.22      0.25        18
                                                        Decree       0.00      0.00      0.00         4
Dedication + dedication to demos + dedication to emperor donor       1.00      0.09      0.17        11
                                                     Factional       0.50      0.50      0.50         2
                                     Funerary + funerary verse       0.43      0.59      0.50        44
                                Gladiator + Gladiator memorial       1.00      0.40      0.57         5
          

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy : 0.45454545454545453
Classification Report :
               precision    recall  f1-score   support

           0       0.67      0.44      0.53         9
           1       0.00      0.00      0.00         2
           2       0.29      0.22      0.25        18
           3       0.00      0.00      0.00         4
           4       1.00      0.09      0.17        11
           5       0.50      0.50      0.50         2
           6       0.43      0.59      0.50        44
           7       1.00      0.40      0.57         5
           8       0.50      0.33      0.40         3
           9       0.48      0.73      0.58        56
          10       0.00      0.00      0.00         2
          11       1.00      0.20      0.33         5
          12       0.22      0.25      0.24         8
          13       0.00      0.00      0.00         6
          14       0.56      0.33      0.42        15
          15       1.00      0.33      0.50         6
          16       0.00  

Les résultats montrent une performance globalement faible avec une précision moyenne autour de 35%. Certaines petites classes affichent de meilleurs scores, mais cela peut être trompeur car elles contiennent très peu d’images, ce qui rend ces métriques peu fiables. En effet, les métriques comme la précision, le rappel ou le F1-score calculées sur de très petites classes ne sont pas toujours fiables, car le nombre réduit d’exemples rend ces mesures très sensibles aux fluctuations. Par exemple, dans une classe avec seulement 5 exemples, prédire correctement 3 d’entre eux peut donner un bon F1-score, mais ce résultat n’est pas très robuste ni représentatif.... Donc on va tenter de faire une validation croisée afin de rendre les résultats un peu plus fiables.

In [ ]:
import timm
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Hyperparamètres
BATCH_SIZE  = 16
IMG_SIZE    = 224
NUM_CLASSES = 18
EPOCHS = 10
K = 4 # nombre de folds pour la validation croisée

# Transformations
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset complet (train + val regroupés pour faire la CV)
dataset = datasets.ImageFolder("/content/drive/MyDrive/Paleo_projet/greekrealdataset", transform=transform)
num_samples = len(dataset)
indices = np.arange(num_samples)

# Configuration device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

kf = KFold(n_splits=K, shuffle=True, random_state=42)

all_fold_metrics = []  # pour stocker les métriques de chaque fold

for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    print(f"Fold {fold+1}")

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)


    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

    # Calcul des poids de classes pour ce fold
    fold_targets = [dataset.targets[i] for i in train_idx]
    class_counts = Counter(fold_targets)
    total_samples = sum(class_counts.values())
    class_weights = [total_samples / class_counts.get(i, 1) for i in range(NUM_CLASSES)]
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    print(f"Poids des classes pour le Fold {fold+1} :", class_weights)

    # Créer un nouveau modèle pour chaque fold
    model = timm.create_model("vit_base_patch16_224", pretrained=True)
    model.head = nn.Linear(model.head.in_features, NUM_CLASSES)
    model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)  # Ajout des poids ici
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

    # Entraînement
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f}")

    # Évaluation à la fin du fold
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f"Résultats Fold {fold+1} - Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    all_fold_metrics.append((acc, precision, recall, f1))

# Moyenne des métriques sur tous les folds
accs, precisions, recalls, f1s = zip(*all_fold_metrics)
print(f"\nMoyenne sur les {K} folds:")
print(f"Accuracy: {np.mean(accs):.4f}, Precision: {np.mean(precisions):.4f}, Recall: {np.mean(recalls):.4f}, F1-score: {np.mean(f1s):.4f}")


Fold 1
Poids des classes pour le Fold 1 : tensor([ 22.6452, 140.4000,  11.3226,  50.1429,  15.9545, 140.4000,   4.5000,
         43.8750, 100.2857,   3.2804, 175.5000,  36.9474,  21.9375,  39.0000,
         14.3265,  30.5217, 234.0000, 702.0000], device='cuda:0')
Epoch 1 | Loss: 3.0762
Epoch 2 | Loss: 2.8404
Epoch 3 | Loss: 2.5664
Epoch 4 | Loss: 1.8891
Epoch 5 | Loss: 0.8285
Epoch 6 | Loss: 0.2620
Epoch 7 | Loss: 0.0675
Epoch 8 | Loss: 0.0237
Epoch 9 | Loss: 0.0426
Epoch 10 | Loss: 0.0088


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Résultats Fold 1 - Accuracy: 0.4170, Precision: 0.4154, Recall: 0.4170, F1-score: 0.3914
Fold 2
Poids des classes pour le Fold 2 : tensor([ 23.4333, 234.3333,  10.9844,  58.5833,  20.0857, 117.1667,   4.3395,
         58.5833,  70.3000,   3.3961, 117.1667,  39.0556,  20.0857,  31.9545,
         11.9153,  41.3529, 140.6000, 703.0000], device='cuda:0')
Epoch 1 | Loss: 2.9922
Epoch 2 | Loss: 2.7187
Epoch 3 | Loss: 2.1618
Epoch 4 | Loss: 1.3871
Epoch 5 | Loss: 0.6891
Epoch 6 | Loss: 0.1526
Epoch 7 | Loss: 0.0294
Epoch 8 | Loss: 0.0083
Epoch 9 | Loss: 0.0038
Epoch 10 | Loss: 0.0027


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Résultats Fold 2 - Accuracy: 0.4359, Precision: 0.4263, Recall: 0.4359, F1-score: 0.4071
Fold 3
Poids des classes pour le Fold 3 : tensor([ 21.9688, 175.7500,  10.6515,  58.5833,  20.6765, 140.6000,   4.0636,
         43.9375,  78.1111,   3.3318, 175.7500,  50.2143,  24.2414,  33.4762,
         14.0600,  39.0556, 140.6000, 703.0000], device='cuda:0')
Epoch 1 | Loss: 2.9604
Epoch 2 | Loss: 2.5034
Epoch 3 | Loss: 1.7705
Epoch 4 | Loss: 0.7152
Epoch 5 | Loss: 0.1832
Epoch 6 | Loss: 0.0290
Epoch 7 | Loss: 0.0087
Epoch 8 | Loss: 0.0040
Epoch 9 | Loss: 0.0026
Epoch 10 | Loss: 0.0021


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Résultats Fold 3 - Accuracy: 0.4487, Precision: 0.4395, Recall: 0.4487, F1-score: 0.4161
Fold 4
Poids des classes pour le Fold 4 : tensor([ 23.4333, 117.1667,  10.6515,  43.9375,  17.5750, 140.6000,   4.2349,
         37.0000, 100.4286,   3.4802, 175.7500,  39.0556,  29.2917,  41.3529,
         12.1207,  35.1500, 140.6000, 703.0000], device='cuda:0')
Epoch 1 | Loss: 3.0007
Epoch 2 | Loss: 2.6739
Epoch 3 | Loss: 2.2312
Epoch 4 | Loss: 1.1497
Epoch 5 | Loss: 0.4691
Epoch 6 | Loss: 0.1369
Epoch 7 | Loss: 0.0202
Epoch 8 | Loss: 0.0054
Epoch 9 | Loss: 0.0031
Epoch 10 | Loss: 0.0024
Résultats Fold 4 - Accuracy: 0.4701, Precision: 0.4439, Recall: 0.4701, F1-score: 0.4464

Moyenne sur les 4 folds:
Accuracy: 0.4429, Precision: 0.4313, Recall: 0.4429, F1-score: 0.4153


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
